# LeNet-5 in Keras

Bulding a Deep Convolutional Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, MaxPooling2D, Conv2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler

from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


#### Load data

In [3]:
n_classes = 10

raw_data = np.loadtxt('kaggle/datasets/train.csv', skiprows=1, dtype='int', delimiter=',')
X_train, X_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size = 0.1)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_val = X_val.astype("float32")/255.

y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)


#### Design Neural Network architecture

In [4]:
model = Sequential()

model.add(Conv2D(32, 5, padding = 'same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(32, 5, padding = 'same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 5, padding = 'same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(64, 5, padding = 'same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

#### Data Augmentation

In [6]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)
datagen.fit(X_train)

#### Learning Rate Scheduler

In [7]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

#### Configure model

In [8]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### TensorBoard

In [9]:
tensorboard = TensorBoard("logs/lenet-cnn-mnist-relu-30-epochs")

#### Train!

In [10]:
hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64),
                           steps_per_epoch = 150,
                           epochs = 30,
                           verbose = 1,
                           validation_data = (X_val[:400,:], y_val[:400,:]),
                           callbacks=[annealer, tensorboard])

Epoch 1/30
150/150 [==============================] - 124s - loss: 0.7990 - acc: 0.7303 - val_loss: 0.1399 - val_acc: 0.9600
Epoch 2/30
150/150 [==============================] - 123s - loss: 0.2593 - acc: 0.9220 - val_loss: 0.1246 - val_acc: 0.9625
Epoch 3/30
150/150 [==============================] - 146s - loss: 0.1810 - acc: 0.9452 - val_loss: 0.0752 - val_acc: 0.9775
Epoch 4/30
150/150 [==============================] - 232s - loss: 0.1408 - acc: 0.9558 - val_loss: 0.0669 - val_acc: 0.9775
Epoch 5/30
150/150 [==============================] - 228s - loss: 0.1297 - acc: 0.9619 - val_loss: 0.0756 - val_acc: 0.9800
Epoch 6/30
150/150 [==============================] - 247s - loss: 0.1028 - acc: 0.9690 - val_loss: 0.0441 - val_acc: 0.9825
Epoch 7/30
150/150 [==============================] - 217s - loss: 0.0971 - acc: 0.9700 - val_loss: 0.0405 - val_acc: 0.9800
Epoch 8/30
150/150 [==============================] - 210s - loss: 0.0886 - acc: 0.9739 - val_loss: 0.0360 - val_acc: 0.9875


#### Test Final Accuracy

In [11]:
final_loss, final_acc = model.evaluate(X_val, y_val, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

4200/4200 [==============================] - 33s    
Final loss: 0.0189, final accuracy: 0.9945


#### Save Submission

In [12]:
X_test = np.loadtxt('kaggle/datasets/test.csv', skiprows=1, dtype='int', delimiter=',')
X_test = X_test.reshape(28000, 28, 28, 1).astype('float32') / 255.

predictions = model.predict(X_test, verbose = 2)
predictions = np.argmax(predictions, axis = 1)

pd.DataFrame({"ImageId": list(range(1, len(predictions) + 1)), "Label": predictions}).to_csv('kaggle/results/cnn/submission-relu-30-epochs.csv', index = False, header = True)